In [38]:
import unicodedata
def remove_tildes(text):
    """Removes the accents of each vowel in a text but preserves the 'ñ'.

    :param str text: The text with vowels with accents.

    :return str: The text without the accents.
    """
    result = []
    for char in text:
        # Preserve 'ñ' and 'Ñ'
        if char in {'ñ', 'Ñ'}:
            result.append(char)
        else:
            # Normalize and remove accents from other characters
            decomposed = unicodedata.normalize('NFD', char)
            filtered = ''.join(c for c in decomposed if unicodedata.category(c) != 'Mn')
            result.append(filtered)
    return ''.join(result)


def format_name(name):
    """
    Format a name by removing any tildes and capitalizing the first letter.

    This function removes tildes (accent marks) from characters in the name and
    then capitalizes the first letter of the name, returning the formatted version.

    :param str name: The name to be formatted.
    :return str: The formatted name with no tildes and capitalized first letter.

    :example:
        >>> format_name('JOSÉ')
        'Jose'
    """
    return remove_tildes(name).lower()

In [48]:
import pandas as pd

to_be_deleted = pd.read_excel('delete_lola.xlsx')
to_be_deleted.drop(['Grado', 'GRUPO', 'Codigo'], axis=1, inplace=True)
to_be_deleted = to_be_deleted[to_be_deleted.isna().any(axis=1)]
to_be_deleted.reset_index(drop=True, inplace=True)

to_be_deleted['Nombre'] = to_be_deleted['Nombre'].map(format_name).str.strip()
to_be_deleted['Nombre']

0          atehortua mira david esteban
1        cañas camargo gloria esperanza
2                       edwin rodriguez
3         estrada quiroz manuel antonio
4                    garcia sierra sara
5               gaviria palacio natalia
6          giraldo orozco jesus huberto
7               giraldo rios juan pablo
8           giraldo villa oscar alberto
9      gutierrez ramirez jose francisco
10       hernandez henao cristian david
11    hernandez morales gustavo alberto
12             juan carlos mesa ramirez
13        londoño arboleda adriana ines
14           londoño zapata luis alonso
15                 maiza abadia bonilla
16     montoya montoya edson alessandro
17                  muñoz diana marcela
18             muñoz zapata julio cesar
19                ospina gomez yennifer
20          padilla castro albeiro jose
21             patricia machado sanchez
22             piedrahita ortiz valeria
23       ramirez mendoza leydi carolina
24        ramirez robledo beatriz elena


In [49]:
downloaded = pd.read_csv('../in/users_lola.csv')
downloaded['Full name R'] = downloaded['Last Name [Required]'] + ' ' + downloaded['First Name [Required]'].str.strip()
downloaded['Full name'] = downloaded['First Name [Required]'] + ' ' + downloaded['Last Name [Required]']
downloaded['Full name'] = downloaded['Full name'].map(format_name)
downloaded['Full name R'] = downloaded['Full name R'].map(format_name)
downloaded

,First Name [Required],Last Name [Required],Email Address [Required],Status [READ ONLY],Last Sign In [READ ONLY],Email Usage [READ ONLY],Full name R,Full name
0,ADALISMAR,GUERRA REINA,adalismar.guerrare@lola.edu.co,Active,2025/02/07 04:06:00,0.0GB,guerra reina adalismar,adalismar guerra reina
1,ALEJANDRO,MONSALVE RENDON,alejandro.monsalvere@lola.edu.co,Active,2025/02/11 03:32:55,0.0GB,monsalve rendon alejandro,alejandro monsalve rendon
2,ALISSON,GARCIA BETANCUR,alisson.garciabe@lola.edu.co,Active,2025/02/10 05:56:19,0.0GB,garcia betancur alisson,alisson garcia betancur
3,ARIANNYS VALENTINA,TORRES VIVAS,arriannys.torresvi@lola.edu.co,Active,Never logged in,0.0GB,torres vivas ariannys valentina,ariannys valentina torres vivas
4,CAMILA VALENTINA,PINEDA BRAVO,camila.pinedabr@lola.edu.co,Active,2025/02/12 07:31:54,0.0GB,pineda bravo camila valentina,camila valentina pineda bravo
...,...,...,...,...,...,...,...,...
4812,ZOE ZAMARA,BOLIVAR URREGO,zoe.bolivarur@lola.edu.co,Active,Never logged in,0.0GB,bolivar urrego zoe zamara,zoe zamara bolivar urrego
4813,ZULY DEL SOCORRO,MUÑOZ BUSTAMANTE,zuly.paz@lola.edu.co,Active,2025/02/14 08:43:43,4.78GB,muñoz bustamante zuly del socorro,zuly del socorro muñoz bustamante
4814,ZURI SARAY,ORREGO ARRIETA,zuri.orregoar@lola.edu.co,Active,2025/02/14 20:39:47,0.54GB,orrego arrieta zuri saray,zuri saray orrego arrieta
4815,Ángela Gabriela,Jaramillo Zuluaga,angela.pta@lola.edu.co,Active,2023/03/03 07:53:37,1.53GB,jaramillo zuluaga angela gabriela,angela gabriela jaramillo zuluaga


In [58]:
# Keep only rows in 'downloaded' where 'Full name' matches 'Nombre' in 'to_be_deleted'
# First merge
deletable_1 = downloaded.merge(to_be_deleted[['Nombre']], left_on='Full name R', right_on='Nombre', how='inner')

# Second merge
deletable_2 = downloaded.merge(to_be_deleted[['Nombre']], left_on='Full name', right_on='Nombre', how='inner')

# Combine both results into one DataFrame
deletable = pd.concat([deletable_1, deletable_2]).drop_duplicates().reset_index(drop=True)


deletable.drop(columns=['Status [READ ONLY]','Last Sign In [READ ONLY]', 'Email Usage [READ ONLY]', 'Full name R', 'Full name', 'Nombre'], inplace=True)
deletable['New Status [Upload Only]'] = 'Suspended'
deletable.to_csv('deletable.csv', index=False)
deletable

,First Name [Required],Last Name [Required],Email Address [Required],New Status [Upload Only]
0,ADRIANA INÉS,LONDOÑO ARBOLEDA,adriana.artistica@lola.edu.co,Suspended
1,ALBEIRO JOSE,PADILLA CASTRO,albeiro.cienciassociales@lola.edu.co,Suspended
2,BEATRIZ ELENA,RAMIREZ ROBLEDO,beatrize.santalucia@lola.edu.co,Suspended
3,Cristian David,Hernández Henao,cristian.religion@lola.edu.co,Suspended
4,Daniel Felipe,Vásquez Franco,daniel.matematicas@lola.edu.co,Suspended
5,david esteban,atehortua mira,david.religion@lola.edu.co,Suspended
6,DORA LUCÍA,RESTREPO RÍOS,doralucia.coordinadora@lola.edu.co,Suspended
7,EDSON ALESSANDRO,MONTOYA MONTOYA,edson.tecnologia@lola.edu.co,Suspended
8,FRANCIS MIGDALIA,RIVERO GIRALDO,francis.santalucia@lola.edu.co,Suspended
9,Gloria Esperanza,Cañas Camargo,gloria.matematicas@lola.edu.co,Suspended


,First Name [Required],Last Name [Required],Email Address [Required],Password [Required],Password Hash Function [UPLOAD ONLY],Org Unit Path [Required],New Primary Email [UPLOAD ONLY],Recovery Email,Home Secondary Email,Work Secondary Email,...,Employee Title,Manager Email,Department,Cost Center,Building ID,Floor Name,Floor Section,Change Password at Next Sign-In,New Status [UPLOAD ONLY],Advanced Protection Program enrollment


In [52]:
# Get rows in 'to_be_deleted' that are NOT in 'deletable'
not_in_deletable = to_be_deleted[~to_be_deleted['Nombre'].isin(deletable['Nombre'])]
not_in_deletable

,Nombre,Correo
2,edwin rodriguez,NaN
4,garcia sierra sara,NaN
5,gaviria palacio natalia,NaN
7,giraldo rios juan pablo,NaN
12,juan carlos mesa ramirez,NaN
17,muñoz diana marcela,NaN
22,piedrahita ortiz valeria,NaN
23,ramirez mendoza leydi carolina,NaN
29,sandra maria mesa,NaN
30,santa silva lisiria,NaN
